In [25]:
# Import dependencies

import subprocess
import json
from dotenv import load_dotenv

In [26]:
# Load and set environment variables
import os 
load_dotenv('ethereum.env')
mnemonic=os.getenv("MNEMONIC_CODE")
#mnemonic ='thought please play lawn camera sail warfare stock invest moral will render'
print(mnemonic)

thought please play lawn camera sail warfare stock invest moral will render


In [27]:
# Import constants.py and necessary functions from bit and web3

# YOUR CODE HERE
from web3 import Web3
from constants import *
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8545"))
from bip_utils import Bip32, Bip39SeedGenerator, Bip44, Bip44Coins, Bip44Changes
from bit import Key, PrivateKey, PrivateKeyTestnet
from bit.network import NetworkAPI
from eth_account import Account
from web3.middleware import geth_poa_middleware
w3.middleware_onion.inject(geth_poa_middleware, layer=0)
seed_bytes = Bip39SeedGenerator(mnemonic).Generate()
seed_bytes

In [56]:
print(constants.ETH)

NameError: name 'constants' is not defined

### Constant py file is properly declared here. But it's working once and other times not working. So I didn't use it.  

In [28]:
# Create a function called `derive_wallets`
def derive_wallets(Mnemonic,Coin,Numderive):
    
    command = f'php derive -g --mnemonic="{Mnemonic}" --numderive={Numderive} --coin={Coin} --format=json'
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    output, err = p.communicate()
    p_status = p.wait()
    return json.loads(output)

In [29]:
# Create a dictionary object called coins to store the output from `derive_wallets`.
coins = {'eth':derive_wallets(Mnemonic=mnemonic,Coin=ETH,Numderive=3), 
  'btc-test': derive_wallets(Mnemonic=mnemonic,Coin=BTCTEST,Numderive=3)}

print(json.dumps(coins, indent=4, sort_keys=True))

{
    "btc-test": [
        {
            "address": "mzWcNgKXqVYoH4cyfhCRcDQ1Z7qT4zuoCw",
            "index": 0,
            "path": "m/44'/1'/0'/0/0",
            "privkey": "cPvEcAengXtLB85U3v1tcnbMowHnXJP42rjUi8jEupFu14wqycrw",
            "pubkey": "02065ff7669f39d9a574a3f54cf1851f155e204859b9c730671f5eb981bfb8302a",
            "pubkeyhash": "d059c2e41d0315e73c11d32bdaf644f66c574566",
            "xprv": "tprv8jEKaRS1uMC9wrEAgJCQdWECCohchQie3r5M91AG9oy9qnd6GyXeD92eCJiCCPWNdQAqezz6E1nvMK89zpLZiovu5tHMYkG4ewzXbm4uxRn",
            "xpub": "tpubDFvMiqUG3ispqKFxZws12utJmqDYrjuYd9g8RXCZa5mYgGsruNMEPdeWNRBBzpgFYbmQ3539GEUa67iNeg2bs9L42mm7gXJMireTGBQQUcr"
        },
        {
            "address": "mhdTvkoLWBGud1T9tvPVp9NVGXv7ewa8Hj",
            "index": 1,
            "path": "m/44'/1'/0'/0/1",
            "privkey": "cRisjkg2m2PxXYWAfDAqfGMRGkxHisSeGjLGmae6Go5DJYwJT7fW",
            "pubkey": "0226d85b6418f634003d1cc09f5afb7a4ba29df926a0d48c3fa546641feea677af",
            "pubkeyh

In [53]:
# Create a function called `priv_key_to_account` that converts privkey strings to account objects.
def priv_key_to_account(coin, priv_key):
    # YOUR CODE HERE
    global account 
    if coin == BTCTEST:
        account=PrivateKeyTestnet(coins['btc-test'][0]['privkey'])
        
    if coin == ETH :
        account=Account.privateKeyToAccount(coins['eth'][0]['privkey'])
    return account

In [39]:
# Create a function called `create_tx` that creates an unsigned transaction appropriate metadata.
def create_tx(coin,account,recipient,amount):
    # YOUR CODE HERE

    global tx_data
    if coin ==ETH:
        gasEstimate = w3.eth.estimateGas(
            {"from": account.address, "to": recipient, "value": amount}
        )
        tx_data= {
            "from": account.address,
            "to": recipient,
            "value": amount,
            "gasPrice":w3.eth.gasPrice,
            "gas": gasEstimate,
            "nonce": w3.eth.getTransactionCount(account.address),
        }
        return tx_data
    else:
        tx_data = PrivateKeyTestnet.prepare_transaction(account.address, [(recipient, amount, BTC)])
        return tx_data

In [40]:
# Create a function called `send_tx` that calls `create_tx`, signs and sends the transaction.

def send_tx(coin, account, recipient, amount):
    if coin == ETH:
        tx = create_tx(coin,account, recipient, amount)
        signed_tx = account.sign_transaction(tx)
        result = w3.eth.sendRawTransaction(signed_tx.rawTransaction)
        print(result.hex())
        return result.hex()
    else:
        tx_data = create_tx(coin,account,recipient,amount)
        signed = account.sign_transaction(tx_data)
        NetworkAPI.broadcast_tx_testnet(signed)
        return signed
    

In [36]:
create_tx(BTCTEST,account,'mxTNhCfimhhGA7t4ZuzkZhVukfZrkswGpY',0.0000000000001)

'{"unspents":[{"amount":552,"confirmations":38,"script":"76a914d059c2e41d0315e73c11d32bdaf644f66c57456688ac","txid":"8bbb66cd7bbe76fec63122ffd1e1cd42447e159ba0c58f1b0c61152c7c448397","txindex":1,"type":"p2pkh","vsize":148,"segwit":false,"sequence":4294967295},{"amount":1239032,"confirmations":1,"script":"76a914d059c2e41d0315e73c11d32bdaf644f66c57456688ac","txid":"1089494f578462731b5081e0337b7eb0be3c46a6afc45808178a30313c1d2395","txindex":1,"type":"p2pkh","vsize":148,"segwit":false,"sequence":4294967295},{"amount":100,"confirmations":38,"script":"76a914d059c2e41d0315e73c11d32bdaf644f66c57456688ac","txid":"8bbb66cd7bbe76fec63122ffd1e1cd42447e159ba0c58f1b0c61152c7c448397","txindex":0,"type":"p2pkh","vsize":148,"segwit":false,"sequence":4294967295},{"amount":100000,"confirmations":0,"script":"76a914d059c2e41d0315e73c11d32bdaf644f66c57456688ac","txid":"097468c3ab25e328fc21ee5868a26021057ac3430700a07cb0be07a2d1843b8b","txindex":0,"type":"p2pkh","vsize":148,"segwit":false,"sequence":429496729

In [37]:
send_tx(BTCTEST,account,'mzWcNgKXqVYoH4cyfhCRcDQ1Z7qT4zuoCw',0.000001)

'010000000495231d3c31308a170858c4afa6463cbeb07e7b33e081501b736284574f498910010000006b483045022100a588162944073fca82178a606386620e2e6dc9e3c74a12d1aaddd4c6fa229a190220438ff28a58bfafcf99c78bdeb0f0cedea50813bd9a65ccf8058db596e429745d012102065ff7669f39d9a574a3f54cf1851f155e204859b9c730671f5eb981bfb8302affffffff9783447c2c15610c1b8fc5a09b157e4442cde1d1ff2231c6fe76be7bcd66bb8b000000006b483045022100d416e8fb4fc45b7caa728628706e51caf23ac3dd7d9729bc38f5c9994bcba10c02203903c7fc1efc514bc6bd8e5391a2e51d29458ac7e3d334eb6dbe76a557d1b2a4012102065ff7669f39d9a574a3f54cf1851f155e204859b9c730671f5eb981bfb8302affffffff9783447c2c15610c1b8fc5a09b157e4442cde1d1ff2231c6fe76be7bcd66bb8b010000006a47304402201b84f96a418bccc1d75c919a0624fc9b155d8648fc8f56560f922b6000a55a8f02205cc588f253c30eb49584bf3805a4336259c497369fbac20ae43ef7e11e1d1f5e012102065ff7669f39d9a574a3f54cf1851f155e204859b9c730671f5eb981bfb8302affffffff8b3b84d1a207beb07ca0000743c37a052160a26858ee21fc28e325abc3687409000000006b483045022100e3d83733853c16031

In [41]:

balance1= w3.eth.getBalance("0x376b0EA75632FdD4962fBf6d15179877db8c707c")
balance2= w3.eth.getBalance("0x50f317823508E458870601Bd07a384E679e75cb7")

print(balance1)
print(balance2)

904625697166532776746648320380374280103671755200316906558262375061821325312
904625697166532776746648320380374280103671755200316906558262375061821325312


In [59]:
print(account.address) 
create_tx(ETH,account,"0x376b0EA75632FdD4962fBf6d15179877db8c707c",104625697166532776746648320380374280103671755200316906558262375061821325312)


0x376b0EA75632FdD4962fBf6d15179877db8c707c


{'from': '0x376b0EA75632FdD4962fBf6d15179877db8c707c',
 'to': '0x376b0EA75632FdD4962fBf6d15179877db8c707c',
 'value': 104625697166532776746648320380374280103671755200316906558262375061821325312,
 'gasPrice': 1000000000,
 'gas': 21000,
 'nonce': 0}

In [60]:
send_tx(ETH,account,'0x376b0EA75632FdD4962fBf6d15179877db8c707c', 104625697166532776746648320380374280103671755200316906558262375061821325312)

ValueError: {'code': -32000, 'message': 'replacement transaction underpriced'}